In [1]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode','geometry'])

In [4]:
#環境省自動車騒音データのダウンロード
car_noise = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/環境省自動車騒音データ/20230526215229/point.csv', encoding='shift-jis')

In [5]:
car_noise

,報告年度,調査自治体名称,一連番号,騒音測定地点番号,測定地点の住所,[1]路線名,[1]車線数,[1]道路種別,[2]路線名,[2]車線数,...,測定終了年月日,車道端距離(m),道路敷地境界距離(m),地上高さ(m),環境基準類型,Leq昼間(dB),Leq夜間(dB),緯度,経度,geometry
0,2008,札幌市,1042,1021,札幌市白石区北郷3条1丁目,北郷1丁目線,4,5,,,...,2008/09/03,3.1,-0.4,1.2,B,67,59,43.0650045120697,141.405555576677,None
1,2010,札幌市,994,1114,札幌市豊平区豊平3条10丁目,一般国道36号,4,3,,,...,2010/09/03,5.0,-0.5,1.2,C,71,69,43.04763008919654,141.37778567457684,None
2,2011,札幌市,1012,1114,札幌市豊平区豊平3条10丁目,一般国道36号,4,3,,,...,2011/09/15,5.0,-0.5,1.2,C,71,69,43.04763,141.377786,None
3,2012,札幌市,261,1114,札幌市豊平区豊平3条10丁目,一般国道36号,4,3,,,...,2012/09/06,5.0,-0.5,1.2,C,71,68,43.04763008919654,141.37778567457684,None
4,2010,札幌市,995,1126,札幌市清田区清田1条1丁目,一般国道36号,4,3,,,...,2010/09/03,4.5,0.0,1.2,C,71,69,43.00119662471074,141.43766077297866,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53778,2015,八重瀬町,3407,40,島尻郡八重瀬町字伊覇,一般国道507号,4,3,,,...,2016/02/18,6.0,0.0,1.2,B,66,59,26.158360569444444,127.72157806111112,None
53779,2015,八重瀬町,3408,41,島尻郡八重瀬町字東風平,一般国道507号,2,3,,,...,2016/02/17,2.6,0.0,1.2,X,70,63,26.148837969444447,127.72447326666668,None
53780,2015,八重瀬町,3409,44,島尻郡八重瀬町字屋宜原,糸満与那原線,2,4,,,...,2016/02/25,2.9,0.0,1.2,X,67,60,26.15875088611111,127.72879145,None
53781,2016,八重瀬町,3457,48,島尻郡八重瀬町字友寄,国道507号,4,3,,,...,2017/03/03,6.8,0.0,1.2,B,68,62,26.16974356388889,127.726101025,None


In [6]:
#カラムを変更する

#現在のカラムを変数に定義
car_noise_columns = car_noise.columns
#新しく設定するカラムを変数に定義
columns_name = ['Report Year',
                'surveyed_municipality',
                'serial_number',
                'Noise_measurement point',
                'Address_of_numbering_point',
                '[1]route_name',
                '[1]number_of_lanes',
                '[1]road_classification',
                '[2]route_name',
                '[2]number_of_lanes',
                '[2]road_classification',
                'Date_of_start_of_measurement',
                'Date_of_measurement_completion',
                'Roadway_edge_distance(m)',
                'Road_site_boundary_distance(m)',
                'Height_above_ground(m)',
                'environmental_standard type',
                'noise_rank_Daytime(dB)',
                'noise_rank_nighttime(dB)',
                'noise_latitude',
                'noise_longitude']
#二つのリストをzip関数でまとめforで一つづつ辞書に入れていく
car_noise = car_noise.rename(columns = {k: v for k, v in zip(car_noise.columns, columns_name)})

In [7]:
#座標系を設定
if car_noise.crs is None:
    # 座標系をEPSG:4326に設定
    car_noise.set_crs(epsg=4326, inplace=True)

#自動車騒音データを奥州市に絞る
car_noise = car_noise[car_noise['Address_of_numbering_point'].str.contains('奥州市', na=False)] #na引数をna=Falseにすることで欠損値を無視することができる

#geometryカラムにpointを付与
car_noise['geometry'] = car_noise.apply(lambda x: Point(x['noise_longitude'], x['noise_latitude']), axis=1) #緯度経度座標の順番が違うと結合することが出来ない。

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
# #ポリゴンデータにポイントデータがあれば空間結合
# car_noise = gpd.sjoin(gdf_osyu_100m_mesh, car_noise, op='contains', how='left')
# #2つのデータフレームで同じカラム（geometry)がないと結合することが出来ない

# #indexが重複しているものを削除する
# car_noise = car_noise[~car_noise.index.duplicated(keep='first')]
# #car_noise_osyuには6行重複があった。

# #indexが重複しているものがないか確認
# print(car_noise[car_noise.index.duplicated(keep=False)])#duplicated(keep=False)とすることで重複しているindexをTrueとしてその行を抜き出す

#結合した際にfiltered_gdf_road_osyuのindexを削除　結合したときのindexを残したまま別のデータフレームは結合できない
car_noise = car_noise.drop(columns=['geometry', 'index_right'], axis=1)

In [11]:
car_noise

,Meshcode,Report Year,surveyed_municipality,serial_number,Noise_measurement point,Address_of_numbering_point,[1]route_name,[1]number_of_lanes,[1]road_classification,[2]route_name,...,Date_of_start_of_measurement,Date_of_measurement_completion,Roadway_edge_distance(m),Road_site_boundary_distance(m),Height_above_ground(m),environmental_standard type,noise_rank_Daytime(dB),noise_rank_nighttime(dB),noise_latitude,noise_longitude
0,5840378485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5840378486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5840378487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5840378495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5840378496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101381,5841728323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101382,5841728324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101383,5841730000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101384,5841730001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
car_noise[~car_noise['noise_latitude'].isna()]

,Meshcode,Report Year,surveyed_municipality,serial_number,Noise_measurement point,Address_of_numbering_point,[1]route_name,[1]number_of_lanes,[1]road_classification,[2]route_name,...,Date_of_start_of_measurement,Date_of_measurement_completion,Roadway_edge_distance(m),Road_site_boundary_distance(m),Height_above_ground(m),environmental_standard type,noise_rank_Daytime(dB),noise_rank_nighttime(dB),noise_latitude,noise_longitude
37455,5841404887,2018,奥州市,250,35,奥州市前沢区,衣川前沢線,2,4,,...,2018/12/20,2018/12/21,2.1,0.0,1.2,X,71,67,39.040516934880664,141.10958048235128
37482,5841404914,2012,奥州市,1759,1,奥州市前沢区新城37,一般国道4号,2,3,,...,2012/11/07,2012/11/08,2.6,0.0,1.2,X,77,73,39.03472426709791,141.11816517500134
38374,5841405806,2013,奥州市,1912,4,奥州市前沢区,衣川前沢線,2,4,,...,2013/11/12,2013/11/13,2.6,0.0,1.2,X,70,62,39.04200811769314,141.10775672440374
39556,5841406988,2007,岩手県,96,28,岩手県奥州市前沢区,一般県道新城馬口沢線,2,4,,...,2007/11/05,2007/11/06,1.7,0.0,1.2,B,65,57,39.057036,141.123666
40497,5841407929,2016,奥州市,260,12,奥州市前沢区,新城馬口沢線,2,4,,...,2016/10/12,2016/10/13,4.7,0.0,1.2,B,63,55,39.06063521944445,141.12476689444443
45684,5841417068,2018,奥州市,249,31,奥州市前沢区古城,一般国道4号,2,3,,...,2018/12/17,2018/12/18,7.3,0.0,1.2,X,67,65,39.064154062040764,141.1356382751731
54495,5841505985,2019,奥州市,256,1,奥州市水沢,東北自動車道,4,1,,...,2019/12/09,2019/12/10,19.5,0.0,1.2,X,62,58,39.13169909939383,141.11949650485425
57803,5841511087,2015,奥州市,262,2,奥州市水沢区真城町下,一般国道4号,2,3,,...,2015/10/13,2015/10/14,3.9,0.0,1.2,B,74,72,39.099076277777776,141.13468598333333
58745,5841512029,2007,岩手県,86,18,岩手県奥州市水沢区真城,一般国道４号,2,3,,...,2007/10/24,2007/10/25,5.9,0.0,1.2,C,72,71,39.10203,141.136348
61807,5841515091,2007,岩手県,95,27,岩手県奥州市水沢区,一般県道衣川水沢線,4,4,,...,2007/11/05,2007/11/06,3.7,0.0,1.2,B,66,59,39.132598,141.126749


In [14]:
car_noise.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　騒音/奥州市_騒音.csv')